# DSE 230: Spark SQL

## Resources

---

Remember: when in doubt, read the documentation first. It's always helpful to search for the class that you're trying to work with, e.g. pyspark.sql.DataFrame.

Spark DataFrame Guide: https://spark.apache.org/docs/latest/sql-programming-guide.html

PySpark API Documentation: https://spark.apache.org/docs/latest/api/python/index.html

Spark SQL Guide: https://spark.apache.org/docs/latest/sql-programming-guide.html

Spark Streaming Guide: https://spark.apache.org/docs/latest/streaming-programming-guide.html

In [ ]:
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StringType, StructType, StructField, IntegerType
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Spark SQL

#### The entry point into all functionality in Spark is the SparkSession class. To create a basic SparkSession, just use SparkSession.builder:

In [ ]:
conf = pyspark.SparkConf().setAll([('spark.master', 'local[*]'),
                                   ('spark.app.name', 'Python Spark SQL Demo')])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

#### Create a DataFrame

In [ ]:
# Create DataFrame based on contents of a JSON file
df = spark.read.json("file:/home/work/people.json")
df.show()

##### Print Schema. Are the types correct?

In [ ]:
<<< YOUR CODE HERE >>>

#### Manually specifying schema by casting

In [ ]:
df = spark.read.text("file:/home/work/people.txt")
df.show()

##### Specify the schema
* Each line is assigned to "value" column
* Split the value column into "name" and "age" columns
* Assign appropriate types to columns. Assign the result to `newDf`
* Useful functions: [DataFrame.withColumn](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.withColumn), [Column.getItem](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.Column.getItem.html), [Column.cast](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.Column.cast.html)

In [ ]:
<<< YOUR CODE HERE >>>

##### Print the schema of `newDf`. Are the types correct?

In [ ]:
<<< YOUR CODE HERE >>>

#### Running SQL queries programatically

The `sql` function on a `SparkSession` enables applications to run SQL queries programmatically and returns the result as a `DataFrame`.

In [ ]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

##### Write an SQL query to print age and name from table `people`
Hint - Use `spark.sql`

In [ ]:
<<< YOUR CODE HERE >>>

#### Infer Schema using Reflection
Spark SQL can convert an RDD of Row objects to a DataFrame, inferring the datatypes. Rows are constructed by passing a list of key/value pairs as kwargs to the Row class. The keys of this list define the column names of the table, and the types are inferred by sampling the whole dataset, similar to the inference that is performed on JSON files.

In [ ]:
# Load a text file and convert each line to a Row.
# Lines is an RDD of rows
lines = spark.sparkContext.textFile("file:/home/work/people.txt")
parts = lines.map(lambda l: l.split(","))

# Define a dictionary of kwargs to specify the schema
# Name is of type str(default) and age is of type int
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

In [ ]:
# Infer the schema
schemaPeople = spark.createDataFrame(people)

# Register the DataFrame as a table.
schemaPeople.createOrReplaceTempView("people")

##### Write SQL query to select the names of all teenagers( 13 <= age <= 19) from the table `people`
Assign the result to `teenagers`

In [ ]:
<<< YOUR CODE HERE >>>

In [ ]:
# DataFrame.rdd returns the content as `pyspark.RDD` of `Row`.
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
for name in teenNames:
    print(name)

#### Programmatically Specifying the Schema


In [ ]:
# Load a text file and convert each line to a Row.
# Lines is an RDD of rows
lines = spark.sparkContext.textFile("file:/home/work/people.txt")
parts = lines.map(lambda l: l.split(","))

# Each line is converted to a tuple.
people = parts.map(lambda p: (p[0], p[1].strip()))

In [ ]:
# Manually specify the schema
fields = [StructField("name", StringType(), True),
         StructField("age", IntegerType(), True)]
schema = StructType(fields)

In [ ]:
# Apply the schema to the RDD.
schemaPeople = spark.createDataFrame(people, schema)

In [ ]:
schemaPeople.printSchema()

In [ ]:
# Creates a temporary view using the DataFrame
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("SELECT name FROM people")

results.show()

In [ ]:
spark.stop()